<h1>Index<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [64]:
import pandas as pd
import numpy as np
import pdb
import requests
import time
import random
import os
import glob
import sys
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from tqdm import tqdm_notebook, tqdm
from joblib import Parallel, delayed

In [65]:
SAVING_PATH = "~/Documents/datasets/cartola"

In [108]:
def go_to_page_number(driver, page_number):
    try:
        driver.find_element_by_link_text(str(page_number)).click()
        time.sleep(10)
        return driver
    except:
        return driver
    

def get_page_numbers(driver):
    soup = BeautifulSoup(driver.page_source, 'lxml')
    tr = soup.find("tr", {"class": "tbpaging"})
    tr = tr.find("tr")
    tds = tr.find_all("td")
    page_numbers = [td.text for td in tds]
    
    return page_numbers

    
def get_table_from_page(driver):
    soup = BeautifulSoup(driver.page_source, 'lxml')

#    table = soup.findAll("table", {"class": "gridView"})
    table = soup.find("table", {"id": "ctl00_cphMainContent_gvList"})
    return pd.read_html(str(table))[0].drop([50, 51])

def input_search_and_click(search, drive):
    return drive

def get_all_tables_from_driver(driver):
    #drive = input_search_and_click(search, driver)
    
    page_numbers = get_page_numbers(driver)
            
    infos = []
#     df = get_table_from_page(driver)
#     infos.append(df)
    
    for page_number in page_numbers:
        go_to_page_number(driver, page_number)
        df = get_table_from_page(driver)
        infos.append(df)

        
    return pd.concat(infos)

# def extract_news_from_page(page):
#     html = requests.get(page).text
#     soup = BeautifulSoup(html)

#     links = soup.find_all("a",
#                          {"class": "feed-post-link"},
#                           href=True)
#     links = [link["href"] for link in links]

#     return links

def get_data_from_championship_round(year, championship_round, search, driver=None):
    page_prefix = "https://www.cartolafcbrasil.com.br/scouts/cartola-fc-{}/rodada-{}"
    page = page_prefix.format(year, championship_round)
    
    if !driver:
        options = Options()
        options.add_argument("--headless")
        driver = webdriver.Firefox(options=options, executable_path="/usr/local/bin/geckodriver")
        driver.implicitly_wait(20)
        
    driver.get(page)
    
    drive = input_search_and_click(search, driver)
    
    data = get_all_tables_from_driver(driver)
    
    return data

In [101]:
# page_prefix = "https://www.cartolafcbrasil.com.br/scouts/cartola-fc-{}/rodada-{}"
# options = Options()
# options.add_argument("--headless")
# driver = webdriver.Firefox(options=options, executable_path="/usr/local/bin/geckodriver")
# driver.implicitly_wait(5)

# year = 2020
# championship_round = 1
# page = page_prefix.format(year, championship_round)
# driver.get(page)
# # df = get_table_from_page(driver)
# # driver.close()

In [102]:
get_maximum_page_number(driver)

['1', '2', '3', '4', '5']

In [103]:
#get_table_from_page(driver)

In [104]:
df = get_data_from_championship_round(2020, 1, {})

In [105]:
df.shape

(238, 25)

In [106]:
df.tail()

,Nome,Clube,Preço,J,Média,Últ. Pont.,Variação,DS,G,A,...,DD,DP,GC,CV,CA,PP,GS,FC,I,PI
25,Carlos Sánchez (MEI),NaN,739,1,-380,-380.0,-661.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,1.0,20.0
26,Filipe Luís (LAT),NaN,384,1,-470,-470.0,-616.0,2.0,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2.0,NaN,12.0
27,Fernando Prass (GOL),NaN,72,1,-660,-660.0,-528.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,6.0
28,12345,12345.0,12345,12345,12345,12345.0,12345.0,12345.0,12345.0,12345.0,...,12345.0,12345.0,12345.0,12345.0,12345.0,12345.0,12345.0,12345.0,12345.0,12345.0
29,1,2.0,3,4,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
df["Nome"].nunique()

229

In [48]:
# driver.find_element_by_link_text("2").click()

In [109]:
get_table_from_page(driver)

,Nome,Clube,Preço,J,Média,Últ. Pont.,Variação,DS,G,A,...,DD,DP,GC,CV,CA,PP,GS,FC,I,PI
0,Ricardo Ryller (MEI),NaN,370,1,310,310.0,170.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2.0,NaN,5.0
1,Allan (MEI),NaN,804,1,310,310.0,4.0,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,3.0,NaN,5.0
2,Cuca (TEC),NaN,873,1,305,305.0,-27.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Gabriel (ATA),NaN,942,1,300,300.0,-58.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
4,Bruno Pacheco (LAT),NaN,725,1,300,300.0,25.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,10.0
5,Pepê (ATA),NaN,797,1,300,300.0,-3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
6,Gabriel (ZAG),NaN,783,1,280,280.0,-17.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2.0,NaN,2.0
7,Guto Ferreira (TEC),NaN,559,1,271,271.0,59.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Nathan Silva (MEI),NaN,413,1,270,270.0,113.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,3.0
9,Rafinha (LAT),NaN,768,1,260,260.0,-32.0,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,9.0
